In [1]:
import pandas as pd
import csv
import numpy as np
import xlrd
from pathlib import Path

In [2]:
vivienda = pd.read_excel('V01BTIPVIV.xlsx')


The function below copies the original df's row by row into a new df if conditions are met.

In [98]:
categories1 = ['Casa', ' Casa indÌgena', ' Apartamento', ' Tipo cuarto', ' Otro tipo de vivienda', ' Total']
categories2 = ['Casos', '%', 'Acumulado %']
index = 'AREA #'
excelcolumns = vivienda.columns
['Tipo_vivo_' + c.replace(' ', '_') + '_' + end.replace(' ', '_') for c in categories1 for end in categories2]
def long_to_wide(df, verticalvar, horizontalvar, columns, index):
    '''
    df: dataframe in long format shown above
    verticalvar: an array of strings - Names of the Row Labels of the Panel Table
    horizontalvar: an array of strings - Names of the Column Labels of the Panel Table
    columns: an array of strings - df columns from the top of the excel sheet
    index: the individual variable that differentiate each panel table
    
    This converts long format excel panel table with index to wide format 
    by filtering through each row and copying to a new dataframe. Thus, it 
    returns a new pandas dataframe with columns that are combinations of 
    verticalvar and horizontalvar.
    '''
    #Compile all the combinations of column names for the output table
    newc = [c + ' '  + end for c in verticalvar for end in horizontalvar]
    
    #Create an empty table with empty columns of the resulting names
    newdf = pd.DataFrame()
    newc.append(index)
    newc.append('Four_digit_num') #Remove or change as needed. This add a column name into the list
    newdf = newdf.reindex(columns = newc)
    
    
    #Creating a dictionary of column names as keys and arrays as the values
    dictionaryofvalues = {}
    for c in newdf.columns:
        dictionaryofvalues[c] = []
        
    ''' Copying the value from the original df into its respective array
    Once it reaches an index, in this case, Area #, it appends an empty value to each 
    newdf's columns.
    In each row, if the condition is true, it will delete the last value 
    and add the new value. This will keep the lengths equal to form a df. 
    This is because not every panel table in the df have the same row labels.'''
    for i, row in df.iterrows():
        if row.isnull().sum() != 4:
            if row.loc[columns[0]] == 'RESUMEN':
                break
            if row.loc[columns[0]][0:6] == index:
                dictionaryofvalues.get(index).append(row[columns[0]][7:])
                dictionaryofvalues.get('Four_digit_num').append(row[columns[1]])
                for kv in range(18): #Change number if needed
#                     print(kv, newc[kv])
                    dictionaryofvalues.get(newc[kv]).append(np.nan)
            if row.loc[columns[0]].replace(' ','') == verticalvar[0]: # Casa
                del dictionaryofvalues.get(newc[0])[-1]
                dictionaryofvalues.get(newc[0]).append(row[columns[1]]) #Casos
                del dictionaryofvalues.get(newc[1])[-1]
                dictionaryofvalues.get(newc[1]).append(row[columns[2]]) # %
                del dictionaryofvalues.get(newc[2])[-1]
                dictionaryofvalues.get(newc[2]).append(row[columns[3]]) #Acumulado %
            if row.loc[columns[0]] == verticalvar[1]: #Casa Indigena
                del dictionaryofvalues.get(newc[3])[-1]
                dictionaryofvalues.get(newc[3]).append(row[columns[1]]) #Casos
                del dictionaryofvalues.get(newc[4])[-1]
                dictionaryofvalues.get(newc[4]).append(row[columns[2]]) # %
                del dictionaryofvalues.get(newc[5])[-1]
                dictionaryofvalues.get(newc[5]).append(row[columns[3]]) #Acumulado %
            if row.loc[columns[0]] == verticalvar[2]: #Apartmento
                del dictionaryofvalues.get(newc[6])[-1]
                dictionaryofvalues.get(newc[6]).append(row[columns[1]]) #Casos
                del dictionaryofvalues.get(newc[7])[-1]
                dictionaryofvalues.get(newc[7]).append(row[columns[2]]) # %
                del dictionaryofvalues.get(newc[8])[-1]
                dictionaryofvalues.get(newc[8]).append(row[columns[3]]) #Acumulado
            if row.loc[columns[0]] == verticalvar[3]: #Tipo cuarto
                del dictionaryofvalues.get(newc[9])[-1]
                dictionaryofvalues.get(newc[9]).append(row[columns[1]]) #Casos
                del dictionaryofvalues.get(newc[10])[-1]
                dictionaryofvalues.get(newc[10]).append(row[columns[2]]) # %
                del dictionaryofvalues.get(newc[11])[-1]
                dictionaryofvalues.get(newc[11]).append(row[columns[3]]) #Acumulado
            if row.loc[columns[0]] == verticalvar[4]: #Otro tipo de vivienda
                del dictionaryofvalues.get(newc[12])[-1]
                dictionaryofvalues.get(newc[12]).append(row[columns[1]]) # Casos
                del dictionaryofvalues.get(newc[13])[-1]
                dictionaryofvalues.get(newc[13]).append(row[columns[2]]) # %
                del dictionaryofvalues.get(newc[14])[-1]
                dictionaryofvalues.get(newc[14]).append(row[columns[3]]) #Acumulado   
            if row.loc[columns[0]] == verticalvar[5]: #Total
                del dictionaryofvalues.get(newc[15])[-1]
                dictionaryofvalues.get(newc[15]).append(row[columns[1]]) #Casos
                del dictionaryofvalues.get(newc[16])[-1]
                dictionaryofvalues.get(newc[16]).append(row[columns[2]]) #%
                del dictionaryofvalues.get(newc[17])[-1]
                dictionaryofvalues.get(newc[17]).append(row[columns[3]]) #Acumulado
        
                
                
    #Adding the arrays into the new dataframe
    for col in newdf.columns:
#           print(col, len(dictionaryofvalues.get(col)))
        newdf[col] = dictionaryofvalues.get(col)
    
    newdf.drop(columns = newdf[newdf.columns[newdf.columns.str.contains('Acumulado')]])
    
    return newdf

output = long_to_wide(vivienda, categories1, categories2, excelcolumns, index)

In [55]:
output.columns = ['Tipo_viv_' + c.replace(' ', '_') for c in output.columns]

In [100]:
output = output.drop(columns = output[output.columns[output.columns.str.contains('Acumulado')]])

In [187]:
output.to_csv('V01BTIPVIV_wideformat_1_8_v2.csv', index=False)

Part 1
1. Delete all the empty rows. Delete the Accumulado % column.
2. Create a new column with Area #, four_digit_num and their values if the row has AREA # and four digit. 

Part 2
3. Fill in the blanks of the new AREA # and new Four_digit_num columns if the row doesn't equal to AREA in column Tipo de vivienda
4. Drop rows that have AREA # in column Tipo de vivienda
5. Drop the rows starting and after "RESUMEN"
6. Keep only the digits of AREA #. Concatenate Area # and Four_digit_num into one ###-##
7. Delete rows that have categories in column Tipo de vivienda

Part 3
8. Rename columns 
9. Groupby index and categories and aggregate its sum. Then pivot on index and columns as categories. Drop the unneccessary resulting columns
10. Rename new column names. Reset index
11. Merge to get four digit nums. Drop duplicates.

This function copies the index number and drag the values down to all rows. Then, pivot the table based on the values (Casa, apartamento, tipo cuarto, etc). 

In [6]:
def long_to_wide_table(df, index, categories):
    #step 1
    df = df.dropna(how='all')
    df = df.drop(columns = ['Unnamed: 3']) #change if necessary
    
    #step 2
    df[categories] = df[categories].astype(str)
    #filter out only the Area # from column categories
    df[index] = df[categories].where(df[categories].str.contains('AREA #'))
    df['Four_digit_num'] = df.apply(
        lambda x: x['Unnamed: 1'] 
        if str(x[categories])[0:6] == 'AREA #' else np.nan, axis = 1) #change if needed
    
    #step 3
    df[index] = df[index].fillna(method = 'ffill')
    df['Four_digit_num'] = df['Four_digit_num'].fillna(method = 'ffill') #change if needed
    
    #step 4
    df = df[~df[categories].str.contains('AREA #')]
    
    #step 5
#     df = df[0:df.shape[0]-resumencount]
    df = df.loc[:(df == 'RESUMEN').any(1).idxmax()-1]

    #step 6
    df[index] = df[index].apply(lambda x: x[7:])
    
    #step 7
    df = df[~df[categories].str.contains('CategorÌas')]
    
    #step 8
    df = df.rename(columns = {'Unnamed: 1': 'casos', 'Unnamed: 2': 'percent'})
    
    #step 9
    newdf = df.groupby(
        [index, categories]).aggregate(
        {'casos': 'sum', 'percent': 'sum'}).reset_index()
    newdf = newdf.pivot_table(
        values = ['casos', 'percent'], columns=categories, index = 'Area #')
    
    newdf = newdf.drop(columns = ['Tabla vacÌa'], axis = 1, level =1)
    
    #step 10
    newdf.columns = ["_".join((i,k)) for i,k in newdf.columns]
    newdf = newdf.reset_index()
    
    #step 11
    return df[[index, 'Four_digit_num']].merge(
        newdf, on = index, how = 'inner').drop_duplicates()
    
# long_to_wide_table(vivienda, 'Area #', 'Tipo de vivienda')

VIVIENDA FOLDER
1. First, turn all the files into xlsx.


In [2]:
viviendo = [str(k) for k in list(Path('Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad').glob('*.xlsx'))]

In [3]:
viviendo

["Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/TOTAL_LEA - Total LEA'S basado en tipo de LEA.xlsx",
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V05B5TELEF - Telefono.xlsx',
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/VC01BTOTHO - Total de Hogares.xlsx',
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V0E1TIPTER - Tipo de territorialidad.xlsx',
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V01BTIPVIV - Tipo de Vivienda.xlsx',
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/VLC01BTIPL - Tipo de LEA.xlsx',
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V05B1ELECT - Energia Electrica.xlsx',
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/VLC02BTOTP - Total de residentes en el LEA.xlsx',
 'Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V05B3ACUED - Acueducto.xlsx',
 'Viviendo - 20 values - Miss

In [7]:
for table in viviendo:
    f = pd.read_excel(table)
    print(table)
    long_to_wide_table(f, 'Area #', f.columns[0]).to_excel('Wide_' + table[67:])

Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/TOTAL_LEA - Total LEA'S basado en tipo de LEA.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V05B5TELEF - Telefono.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/VC01BTOTHO - Total de Hogares.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V0E1TIPTER - Tipo de territorialidad.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V01BTIPVIV - Tipo de Vivienda.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/VLC01BTIPL - Tipo de LEA.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V05B1ELECT - Energia Electrica.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/VLC02BTOTP - Total de residentes en el LEA.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialidad/V05B3ACUED - Acueducto.xlsx
Viviendo - 20 values - Missing V0E2CODTER, Condigo territorialida

In [71]:
hogar = [str(k) for k in list(Path('Hogar').glob('*.xlsx'))]

In [77]:
for table in hogar:
    f = pd.read_excel(table)
    print(table)
    long_to_wide_table(f, 'Area #', f.columns[0]).to_excel('Wide_' + table[6:])

Hogar/$TOTAL_HOG - Total HOgares Basado en Nro del hogar.xlsx
Hogar/$HC06BTOTPE - Total personas hogar.xlsx
Hogar/$TOTELD - Total de personas de 65+.xlsx
Hogar/$HC07BACTEC - Hay Actividad economica hogar.xlsx
Hogar/$NROUPZH - Nro de UPZ Hogar.xlsx
Hogar/$H0ANROHOGA - Numero Hogar.xlsx


In [80]:
persona = [str(k) for k in list(Path('Persona').glob('*.xlsx'))]
for table in persona:
    f = pd.read_excel(table)
    print(table)
    long_to_wide_table(f, 'Area #', f.columns[0]).to_excel('Wide_' + table[8:])

Persona/$P30B5ANOLL - Anos en que llego al pais hace 5 anos.xlsx
Persona/$EDDECE - Edad en grupos decenales.xlsx
Persona/$P32B4DPTOU - Departamento del ultimo cambio - Bogota.xlsx
Persona/$P30B0DPMP5 - DPto_Mpto donde vivia hace 5 anos - MPIOS.xlsx
Persona/$PC08B1NROP - Numero Orden Persona - MPIOS.xlsx
Persona/$P32B7PAISU - Pais del ultimo cambio.xlsx
Persona/$P35B1AYUNO - Tuvo dias de ayuno.xlsx
Persona/$P28B5ANOLL - Ano en que llego al pais.xlsx
Persona/$P32B8CAUSA - Causa del cambio del lugar residencia.xlsx
Persona/$P32B6CLASE - Clase dentro del municpio.xlsx
Persona/$TIPO_ESTUD - Tipo de estudios que curso - MPIOS.xlsx
Persona/$P30B3MPIO5 - Municipio donde vivia hace 5 anos - MPIOS.xlsx
Persona/$P30B4PAIS5 - Pais donde vivia hace 5 anos.xlsx
Persona/$P32B5MPIOU - Municipio del ultimo cambio- MPIOS.xlsx
Persona/$P32B1CAMBI - Cambio de lugar ultimos 5 anos.xlsx
Persona/$NIVEL_ANO    - Nivel y anos aprobado - Bogota.xlsx
Persona/$TOTAL_PER - Total de personas.xlsx
Persona/$P44B2ULTA